# A hands-on guide to LLM-based AI Agents
# Solutions to Exercise Block 1
#### ML Prague 2025
#### Philipp Wendland <pwendland@deloitte.de>
This notebook contains exercises for the workshop at ML Prague 2025. The goal is to implement an AI Agent that can assist you in exploring a new city and planning various trips according to your preferences.
***
###### Sources: 
- https://huggingface.co/blog/smolagents
- https://huggingface.co/docs/smolagents/index
- https://github.com/anthropics/anthropic-cookbook/
- https://platform.openai.com/docs/overview
***

## Exercise 1: Simple Prompt
This exercise will ensure your environment, API keys and accesses are configured correctly. Additionally you will create a simple prompt to use as a reference point when building your agent.

Complete the following items:
- [ ] Set up API key
- [ ] Adjust the prompt to your preferences
- [ ] Examine the simple output from the LLM

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Either enter your API key directly here
# client = OpenAI(api_key="***")
# or use dotenv to set it as an environment variable (or simply export it)
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

model = "gpt-4o-mini"
system_prompt = "Be a helpful assistant"
city = "Prague"

prompt = "Act as a professional travel planner. I'm staying in {0} for the Machine Learning Prague conference.\
During the day I will attend seminars and talks, but in the evening I'm mostly free. Plan an itinerary for must-see things\
in {0} for Monday and Tuesday night. I'm interested in both night-life and culture.".format(city)

def llm_call_openai(prompt: str, model="gpt-4o-mini") -> str: 
    """
        Calls the specified OpenAI model with the given prompt
        Returns: 
            str: Completion from the LLM
    """

    response = client.responses.create(
        model=model,
        instructions=system_prompt,
        input=prompt
    )

    return response.output[0].content[0].text

completion = llm_call_openai(prompt, model)

print(completion)

Absolutely, Prague is a beautiful city with a rich cultural scene and vibrant nightlife. Here’s a tailored itinerary for your evenings after attending the Machine Learning Prague conference on Monday and Tuesday.

### Monday Evening

**6:00 PM - Dinner at Lokál Dlouhááá**
- Enjoy a traditional Czech meal in a lively beer hall. Lokál offers authentic Czech cuisine and a variety of local beers. 

**7:30 PM - Vltava River Cruise**
- Take a scenic evening cruise along the Vltava River. Enjoy the stunning views of Prague Castle, Charles Bridge, and the city skyline illuminated at night. Many cruises include dinner and live music.

**9:30 PM - Explore the Old Town Square**
- Stroll through the Old Town Square (Staroměstské náměstí) to see the Astronomical Clock and Týn Church beautifully lit up at night. Take your time to enjoy the vibrant atmosphere with street musicians and performers.

**10:30 PM - Cocktail at Hemingway Bar**
- End your night at this renowned cocktail bar, known for its c

## Exercise 2: Prompt Workflow
Create a prompt workflow (aka prompt-chain) to break down the task of planning an itinerary into multiple steps. The `sequential_chain` method always passes the output of the previous generation between steps. In the first step `add_information` is passed.

Complete the following items:
- [ ] Edit the list of prompts `create_itinerary_prompts`, add at least one more sequential prompt. Compare the output to the results from Exercise 1
- [ ] Write the `additive_chain` function, where all previous generated output is passed between steps, change the prompt-list if necessary and re-run the chain
- [ ] **BONUS**: Try a different use case: e.g., create a marketing pitch for the sight-seeing tour and translate it to a different language / tonality.

In [3]:
def sequential_chain(prompts, input = "") -> str:
    """Create a sequential chain of prompts, where only the result is passed between steps."""
    completion = input
    for i, prompt in enumerate(prompts, 1):
        print("Step {}:".format(i))
        completion = llm_call_openai("{}\n###\nGiven information: {}\n###\n".format(prompt, completion))
        print(completion)
    return completion


create_itinerary_prompts = [
    """Act as a professional travel planner. I'm staying in Prague for the Machine Learning Prague conference.
    During the day I will attend seminars and talks, but in the evening I'm mostly free. Create a list of items that
    could potentially be interesting. Be creative.""",
    
    """Plan an itinerary for must-see things in Prague for Monday and Tuesday night.""",
]

add_information = "I'm interested in both night-life and culture."

sequential_chain(create_itinerary_prompts, add_information)


Step 1:
Certainly! Here’s a list of things to do in Prague at night that balances nightlife and cultural experiences:

### Nightlife

1. **Jazz Clubs**: 
   - **JazzDock**: A modern jazz club right on the river, offering live performances.
   - **Reduta Jazz Club**: Famous for hosting international jazz artists and a cozy atmosphere.

2. **Rooftop Bars**:
   - **Twelve Apostles**: Enjoy stunning views of the city while sipping delicious cocktails.
   - **Sky Bar**: Located in the City Centre, great for panoramic views and a vibrant atmosphere.

3. **Historic Pubs**:
   - **U Fleků**: A famous brewery and pub, known for its dark lager and vibrant ambiance.
   - **Lokál**: A traditional Czech pub with a fantastic atmosphere and authentic local food.

4. **Night Clubs**:
   - **Radost FX**: A popular nightclub known for its eclectic music styles and a vibrant international crowd.
   - **Roxy**: A cultural institution featuring club nights and live music.

5. **River Cruise**: 
   - Evenin

"Here’s a two-night itinerary for must-see experiences in Prague that balances nightlife and cultural activities for Monday and Tuesday.\n\n### Monday Night: Cultural Evening & Relaxation\n\n**6:00 PM - Evening Walk at Charles Bridge**\n- Start your evening with a scenic walk along the iconic Charles Bridge. Enjoy the statues and the views of the Vltava River as the sun begins to set.\n\n**7:00 PM - Dinner at Lokál**\n- Head to Lokál for a traditional Czech dinner. Enjoy authentic dishes and local beer in a bustling yet cozy atmosphere.\n\n**8:30 PM - Night Tour of Prague Castle**\n- Join a guided night tour of Prague Castle. It's a unique experience to explore the area after hours, with fewer crowds and stunning illuminated views.\n\n**10:00 PM - Jazz at Reduta Jazz Club**\n- Finish your night at Reduta Jazz Club. Enjoy live jazz music in a vibrant setting. Be sure to check the schedule for any special performances.\n\n### Tuesday Night: Lively Atmosphere & Unique Experiences\n\n**6:0

In [4]:
def additive_chain(prompts, input = "") -> str:
    """Create an additive chain of prompts, where all previous input and output is passed between steps."""
    
    # TODO
    history = ""
    # START SOLUTION
    initial_prompt = "{}\n###\nGiven information: {}\n###\n".format(prompts[0], input)
    history += "User:\n" + initial_prompt
    completion = llm_call_openai(initial_prompt)
    history += "\nSystem:\n" + completion

    print("Step 1:")
    print(completion)
    
    for i, prompt in enumerate(prompts[1:], 2):
        print("Step {}:".format(i))
        completion = llm_call_openai(history + "\n" + prompt)
        history += "\n\nUser:\n" + prompt + "\nSystem:\n" + completion

        print(completion)

    # END SOLUTION
    return history
    

create_itinerary_prompts_additive = [
    """Act as a professional travel planner. I'm staying in Prague for the Machine Learning Prague conference.
    During the day I will attend seminars and talks, but in the evening I'm mostly free. Create a list of items that
    could potentially be interesting. Be creative""",
    
    """Plan an itinerary for must-see things in Prague for Monday and Tuesday night."""
]

print("\n\nComplete interaction\n{}".format(additive_chain(create_itinerary_prompts_additive, add_information)))

Step 1:
Absolutely! Prague is a vibrant city with a rich cultural scene and an exciting nightlife. Here’s a list of suggestions for activities you could explore during your evenings in Prague. 

### Cultural Activities:
1. **Old Town Walking Tour**: Stroll through the historic Old Town Square, marvel at the Astronomical Clock, and enjoy the gothic architecture.
   
2. **Visit the Prague Castle**: Evenings can be less crowded; walk through the gardens and enjoy stunning views of the city illuminated at night.

3. **Vltava River Cruise**: Enjoy a relaxing evening boat cruise with dinner options, offering sweeping views of the city's skyline and a unique perspective on iconic landmarks.

4. **Attend a Classical Concert**: Check local venues like the Rudolfinum or the National Theatre for classical music performances.

5. **Explore the Jewish Quarter**: Visit the synagogues and the Jewish Museum, many of which have evening tours available.

6. **Museum of Alchemy**: Discover the mystical w

**Bonus**: Try a different use case: e.g., create a marketing pitch for the sight-seeing tour and translate it to a different language / tonality.

In [5]:
marketing_pitch_prompts = [
    # START SOLUTION
    """Act as a professional travel planner. I'm staying in Prague for the Machine Learning Prague conference.
    During the day I will attend seminars and talks, but in the evening I'm mostly free. Create a list of items that
    could potentially be interesting. Be creative""",
    
    """Plan an itinerary for must-see things in Prague for Monday and Tuesday night. """,

    """Act as a professional marketing agency. Create a quick marketing advertisement text for the itnerary.
    Your target audience are Machine Learning enthusiasts.""",

    """Rewrite the given text in pirate speech. Stay as close to the original as possible"""
    # END SOLUTION
]

print(sequential_chain(marketing_pitch_prompts))


Step 1:
Absolutely, Prague is a beautiful city with a rich history, vibrant culture, and plenty to explore in the evenings. Here’s a curated list of interesting activities you can enjoy after your conference seminars and talks:

### Evening Activities in Prague:

1. **Vltava River Cruise**  
   - Experience a scenic dinner cruise on the Vltava River, offering beautiful views of Prague’s illuminated skyline, including the iconic Charles Bridge and Prague Castle.

2. **Old Town Square and Astronomical Clock**  
   - Visit the historical Old Town Square and watch the Astronomical Clock's hourly show. The ambiance of the square in the evening is magical.

3. **Prague Castle Night Tour**  
   - Take a guided night tour of Prague Castle for a unique perspective on its architectural beauty and history when the crowds have thinned.

4. **Opera or Ballet at the National Theatre**  
   - Check the schedule for an evening performance. The National Theatre is renowned for its beautiful productions

## Exercise 3a: Tool Calling
Before introducing the smolagents framework, we will implement a tool call using standard OpenAI functionalities. We will add the option for tool calls into the prompt-chain.

- [ ] Examine the implemented tool call for obtaining current weather information.
    - [ ] Extend both `llm_call_openai` and `sequential_chain` to be able to handle tool calls
    - [ ] Add the tool call in the beginning of the prompt chain to add current weather information to the planning.

In [14]:
# For additional documentation refer to: https://platform.openai.com/docs/guides/function-calling#sample-function
# open-meteo docs: https://open-meteo.com/en/docs
import requests
import json

def get_current_weather(latitude=50.073658, longitude=14.418540):
    try: 
        response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,cloud_cover,precipitation&hourly=temperature_2m,precipitation_probability,cloud_cover")
    except: 
        return "Weather data not available"
    response_data = response.json()
    return {
        "current_temp": response_data['current']['temperature_2m'],
        "cloud_cover": response_data['current']['cloud_cover'],
        "precipitation": response_data['current']['precipitation']
    }


tools = [{
    "type": "function",
    "name": "get_current_weather",
    "description": "Get current weather for provided coordinates, including the temperature in celsius, the cloud cover percentage and the precipitation in mm.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

prompt = "What is the weather like in Melbourne today?"

response = client.responses.create(
    model="gpt-4o-mini",
    input=prompt,
    tools=tools,
)
print(response)
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

current_weather = get_current_weather(args['latitude'], args["longitude"])

print("\nTool output:\n{}".format(current_weather))

Response(id='resp_680b9403a4388191a9b66547ed5cb9f60d3c945c98f04376', created_at=1745589251.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFunctionToolCall(arguments='{"latitude":-37.8136,"longitude":144.9631}', call_id='call_WC3POtm3NAkBJmGyIWt5I3k4', name='get_current_weather', type='function_call', id='fc_680b94045e988191bdd15f6e0f87cb1e0d3c945c98f04376', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_current_weather', parameters={'type': 'object', 'properties': {'latitude': {'type': 'number'}, 'longitude': {'type': 'number'}}, 'required': ['latitude', 'longitude'], 'additionalProperties': False}, strict=True, type='function', description='Get current weather for provided coordinates, including the temperature in celsius, the cloud cover percentage and the precipitation in mm.')], top_p=1.0, max_output_tokens=None, previous_re

In [15]:
# Helper function to accomodate for multiple tool calls
def call_function(name, args): 
    if name == "get_current_weather":
        return get_current_weather(**args)


def llm_call_openai_tools(prompt: str, tools=None, model="gpt-4o-mini") -> str: 
    """
        Calls the specified OpenAI model with the given prompt and if specified uses the given tools
        Returns: 
            str: Completion from the LLM or result from the tool call
    """   

    response = client.responses.create(
        model=model, 
        input=prompt,
        tools=tools,
    )
    
    if tools:
        tool_call_responses = []
        for tool_call in response.output:

            tool_name = tool_call.name
            args = json.loads(tool_call.arguments)
            tool_call_responses.append([prompt, call_function(tool_name, args)])
        
        return tool_call_responses

    return response.output[0].content[0].text

def sequential_chain_with_tools(prompts, input = "") -> str:
    """Create a sequential chain of prompts, where only the result is passed between steps.
    Prompts now is a list of tuples that can contain a list of tools"""
    completion = input
    for i, prompt in enumerate(prompts, 1):
        print("Step {}:".format(i))
        if isinstance(prompt, list):
            completion = llm_call_openai_tools(prompt[0], prompt[1])
        else:
            completion = llm_call_openai_tools("{}\n###\nGiven information: {}\n###\n".format(prompt, completion))
        print(completion)
    return completion

create_itinerary_prompts_with_tools = [
    ["""What is the current weather in Prague""", tools],
    
    """Act as a professional travel planner. I'm staying in Prague for the Machine Learning Prague conference.
    During the day I will attend seminars and talks, but in the evening I'm mostly free. Create a list of items that
    could potentially be interesting. Be creative""",

    """Plan an itinerary for must-see things in Prague for Monday and Tuesday night."""
]

sequential_chain_with_tools(create_itinerary_prompts_with_tools)

# END SOLUTION

Step 1:
[['What is the current weather in Prague', {'current_temp': 11.2, 'cloud_cover': 100, 'precipitation': 0.1}]]
Step 2:
That sounds like an exciting trip! Here’s a list of interesting evening activities in Prague to explore during your free time, especially considering the current weather and your location at the Machine Learning Prague conference:

### Evening Activities in Prague

1. **Vltava River Cruise**  
   Enjoy a leisurely dinner cruise on the Vltava River. You’ll get magnificent views of illuminated landmarks like Prague Castle and Charles Bridge.

2. **Explore the Old Town Square**  
   Take a stroll around the Old Town Square. You can admire the Astronomical Clock, listen to street musicians, and enjoy the vibrant atmosphere.

3. **Visit a Traditional Czech Restaurant**  
   Savor authentic Czech cuisine at a local restaurant. Consider trying goulash, svíčková, or trdelník for dessert.

4. **Jazz Clubs or Live Music**  
   Experience Prague’s vibrant jazz scene at a l

"Here's a two-day itinerary for your evenings in Prague, highlighting must-see activities and experiences for both Monday and Tuesday nights:\n\n### **Monday Night Itinerary**\n\n1. **6:00 PM - Vltava River Cruise**\n   - Kick off your evening with a dinner cruise on the Vltava River. Enjoy stunning views of illuminated landmarks such as Prague Castle and the Charles Bridge while savoring traditional Czech cuisine.\n\n2. **8:30 PM - Walk Across Charles Bridge**\n   - After the cruise, take a leisurely stroll across Charles Bridge. The atmosphere is magical at dusk with street artists and musicians enhancing your experience.\n\n3. **9:00 PM - Visit the Lennon Wall**\n   - Head to the Lennon Wall, which is vibrant and colorful, especially as the sun sets. Take some time to appreciate the art and the messages of peace and love.\n\n4. **9:30 PM - Jazz Club**\n   - Finish the night at a jazz club like JazzDock or Reduta. Enjoy live music in a cozy setting to immerse yourself in Prague’s ric